In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
import os

In [2]:
spark = (
    SparkSession.builder
    .appName("Exercise6-Optimized")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
)
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/03 11:03:06 WARN Utils: Your hostname, developer, resolves to a loopback address: 127.0.1.1; using 192.168.29.13 instead (on interface enp2s0)
25/11/03 11:03:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/03 11:03:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
csv_files_path = "csv_files"
# Check if the folder exists
if not os.path.exists(csv_files_path):
    raise FileNotFoundError(f"Folder not found: {csv_files_path}")
else:
    print(f"Found CSV folder: {csv_files_path}")

Found CSV folder: csv_files


In [4]:
df = spark.read.csv("csv_files/Divvy_Trips_2019_Q4.csv",header=True,inferSchema=True)
print("Data loaded successfully!")

Data loaded successfully!


In [5]:
df.count()

704054

In [6]:
df.show()

+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|         start_time|           end_time|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|  2215|       940.0|             20|Sheffield Ave & K...|          309|Leavitt St & Armi...|Subscriber|  Male|     1987|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|  6328|       258.0|             19|Throop (Loomis) S...|          241| Morgan St & Polk St|Subscriber|  Male|     1998|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|  3003|       850.0|             84|Milwaukee Ave & G...|          199|Wabash Ave & Gran...|Sub

In [7]:
df.write.partitionBy("from_station_name").mode("overwrite").csv("partition_files")

In [9]:
df.rdd.getNumPartitions()

4

In [11]:
num_partitions = len(df.rdd.glom().collect())
print("Number of partitions:", num_partitions)

Number of partitions: 4


In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [14]:
path = "partition_files"

fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(sc._jsc.hadoopConfiguration())
p = sc._jvm.org.apache.hadoop.fs.Path(path)

# Get all subdirectories (i.e. partitions)
statuses = fs.listStatus(p)

partition_dirs = [f.getPath().getName() for f in statuses if f.isDirectory()]

print(f"Total number of partitions: {len(partition_dirs)}")

Total number of partitions: 610
